### Quick Start

### AF-TODO: Change badge branch
[![Open with marimo](https://marimo.io/shield.svg)](https://molab.marimo.io/github/lnccbrown/ssm-simulators/blob/first-marimo-tutorial/docs/basic_tutorial/basic_tutorial.ipynb)
[![Open with colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lnccbrown/ssm-simulators/blob/first-marimo-tutorial/docs/basic_tutorial/basic_tutorial.ipynb)

The `ssms` package serves two purposes. 

1. Easy access to *fast simulators of sequential sampling models*
   
2. Support infrastructure to construct training data for various approaches to likelihood / posterior amortization

We provide two minimal examples here to illustrate how to use each of the two capabilities.

#### Install 

Let's start with *installing* the `ssms` package.

You can do so by typing,

`pip install ssm-simulators`

in your terminal.

Below you find a basic tutorial on how to use the package.

#### Notebook Setup

In [1]:
import os

if ("COLAB_RELEASE_TAG" in os.environ):
    !pip install ssm-simulators
    pass # helps keep code running in molab
elif any(["MARIMO" in item_ for item_ in os.environ]):
    import marimo as mo
    mo.dependencies(pip=["ssm-simulators", "pandas"])

#### Tutorial

In [2]:
# Import necessary packages
import numpy as np
import pandas as pd
import ssms

#### Using the Simulators

Let's start with using the basic simulators. 
You access the main simulators through the  `ssms.basic_simulators.simulator.simulator()` function.

To get an idea about the models included in `ssms`, use the `config` module.
The central dictionary with metadata about included models sits in `ssms.config.model_config`. 

In [3]:
# Check included models
list(ssms.config.model_config.keys())[:10]

['ddm',
 'ddm_st',
 'ddm_truncnormt',
 'ddm_rayleight',
 'ddm_sdv',
 'ddm_par2',
 'ddm_par2_no_bias',
 'ddm_par2_conflict_gamma_no_bias',
 'ddm_par2_angle_no_bias',
 'ddm_par2_weibull_no_bias']

In [4]:
# Take an example config for a given model
ssms.config.model_config["ddm"]

{'name': 'ddm',
 'params': ['v', 'a', 'z', 't'],
 'param_bounds': [[-3.0, 0.3, 0.1, 0.0], [3.0, 2.5, 0.9, 2.0]],
 'boundary_name': 'constant',
 'boundary': <function ssms.basic_simulators.boundary_functions.constant(t: float | numpy.ndarray = 0, a: float = 1.0) -> float | numpy.ndarray>,
 'boundary_params': [],
 'n_params': 4,
 'default_params': [0.0, 1.0, 0.5, 0.001],
 'nchoices': 2,
 'choices': [-1, 1],
 'n_particles': 1,
 'simulator': <cyfunction ddm_flexbound at 0x125bd7ac0>,
 'param_bounds_dict': {'v': (-3.0, 3.0),
  'a': (0.3, 2.5),
  'z': (0.1, 0.9),
  't': (0.0, 2.0)}}

**Note:**
The usual structure of these models includes,

- Parameter names (`'params'`)
- Bounds on the parameters (`'param_bounds'`)
- A function that defines a boundary for the respective model (`'boundary'`)
- The number of parameters (`'n_params'`)
- Defaults for the parameters (`'default_params'`)
- The number of choices the process can produce (`'nchoices'`)

The `'hddm_include'` key concerns information useful for integration with the [hddm](https://github.com/hddm-devs/hddm) python package, which facilitates hierarchical bayesian inference for sequential sampling models. It is not important for the present tutorial.

In [5]:
from ssms.basic_simulators.simulator import simulator

sim_out = simulator(
    model="ddm", theta={"v": 0, "a": 1, "z": 0.5, "t": 0.5}, n_samples=1000,
)

The output of the simulator is a `dictionary` with three elements.

1. `rts` (array)
2. `choices` (array)
3. `metadata` (dictionary)

The `metadata` includes the named parameters, simulator settings, and more.

#### Using the Training Data Generators

The training data generators sit on top of the simulator function to turn raw simulations into usable training data for training machine learning algorithms aimed at posterior or likelihood amortization.

We will use the `TrainingDataGenerator` class from `ssms.dataset_generators.lan_mlp`. Initializing the `TrainingDataGenerator` boils down to supplying a configuration dictionary.

The `config` dictionary concerns choices as to what kind of training data one wants to generate, including the underlying generative *sequential sampling model*. 

We will consider a basic example here, concerning data generation to prepare for training [LANs](https://elifesciences.org/articles/65074).

Let's start by peeking at an example `config`. The config is organized into sections: `pipeline`, `simulator`, `estimator`, `training`, and `output`.

In [6]:
from ssms.config.generator_config.data_generator_config import get_lan_config

# Get LAN config (returns nested dictionary)
config = get_lan_config()

# Show the nested structure
print("Config sections:", config.keys())
print("\nPipeline settings:", config["pipeline"])
print("\nSimulator settings:", {k: config["simulator"][k] for k in ["n_samples", "max_t", "delta_t"]})

Config sections: dict_keys(['pipeline', 'estimator', 'training', 'simulator', 'output', 'model'])

Pipeline settings: {'n_parameter_sets': 10000, 'n_parameter_sets_rejected': 100, 'n_subruns': 10, 'n_cpus': 'all', 'simulation_filters': {'mode': 20, 'choice_cnt': 0, 'mean_rt': 17, 'std': 0, 'mode_cnt_rel': 0.95}}

Simulator settings: {'n_samples': 100000, 'max_t': 20.0, 'delta_t': 0.001}


You usually have to make just few changes to this basic configuration dictionary.
An example below.

In [7]:
from copy import deepcopy

# Initialize the generator config (for MLP LANs)
generator_config = deepcopy(config)

# Specify number of parameter sets to simulate
generator_config["pipeline"]["n_parameter_sets"] = 100

# Specify how many samples a simulation run should entail
generator_config["simulator"]["n_samples"] = 1000

Now let's define our corresponding `model_config`.

In [8]:
model_config = ssms.config.model_config["angle"]
print(model_config)

{'name': 'angle', 'params': ['v', 'a', 'z', 't', 'theta'], 'param_bounds': [[-3.0, 0.3, 0.1, 0.001, -0.1], [3.0, 3.0, 0.9, 2.0, 1.3]], 'boundary_name': 'angle', 'boundary': <function angle at 0x120697380>, 'n_params': 5, 'default_params': [0.0, 1.0, 0.5, 0.001, 0.0], 'nchoices': 2, 'choices': [-1, 1], 'n_particles': 1, 'simulator': <cyfunction ddm_flexbound at 0x125bd7ac0>, 'param_bounds_dict': {'v': (-3.0, 3.0), 'a': (0.3, 3.0), 'z': (0.1, 0.9), 't': (0.001, 2.0), 'theta': (-0.1, 1.3)}}


We are now ready to initialize a `TrainingDataGenerator`, after which we can generate training data using the `generate_data_training` function, which will use the hypercube defined by our parameter bounds from the `model_config` to uniformly generate parameter sets and corresponding simulated datasets.

In [9]:
from ssms.dataset_generators.lan_mlp import TrainingDataGenerator

my_dataset_generator = TrainingDataGenerator(
    config=generator_config, model_config=model_config
)

In [10]:
training_data = my_dataset_generator.generate_data_training(save=False)

`training_data` is a dictionary containing four keys:

1. `data` the features for [LANs](https://elifesciences.org/articles/65074), containing vectors of *model parameters*, as well as *rts* and *choices*.
2. `labels` which contain approximate likelihood values
3. `generator_config`, as defined above
4. `model_config`, as defined above

You can now use this training data for your purposes. If you want to train [LANs](https://elifesciences.org/articles/65074) yourself, you might find the [LANfactory](https://github.com/AlexanderFengler/LANfactory) package helpful.

You may also simply find the basic simulators provided with the **ssms** package useful, without any desire to use the outputs into training data for amortization purposes.

##### END